# The Drunkard's Walk In Detail

## Introduction

I would like to spend some time on a simple random walk on the non-negative integers <code>0 ... k</code> called "the drunkard's walk"  A *lot* of intuition can be had by studying the this system. This note is a quicker way to derive some of the observations found in ["A Slightly Unfair Game"](https://win-vector.com/2023/10/30/a-slightly-unfair-game/).

I have found this differs from many treatments of Markov chains in that we are analyzing an "absorbing chain" (instead of a "regular" or "ergodic" one, see Kemeny, Snell, *Finite Markov Chains*, 2nd Edition, 1976, Springer Verlag) and we are using bespoke recurrence arguments instead of the usual move to linear algebra.


## The drunkard's walk

Pick an integer <code>k &gt; 0</code>. Then the states of our random process are the integers <code>0</code> through <code>k</code>. The integers <code>0</code> and <code>k</code> are both "stop conditions" called "stop zero" and "stop non-zero". For an integer <code>i</code> strictly larger than <code>0</code> and strictly less than <code>k</code>, our random process is: pick the next integer to be either <code>i-1</code> or <code>i+1</code> with equal probability. We stop the process when <code>i</code> is either <code>0</code> or <code>k</code>. This is called a "random walk" and is often known as "the drunkard's walk". Some variations have "stay probabilities", but we will not need these here.

What isn't emphasized enough is: a lot can be quickly seen for this walk, without using complicated tools or arguments. In particular we can quickly show the following.

  1) The probability of a walk started in state-</code>i</code> "stopping non-zero" (first reaching <code>k</code>, with no prior visits to <code>0</code>) is <code>i/k</code>.
  2) The expected time of a walk started in state-</code>i</code> to hit either of the stopping conditions (reaching <code>0</code> or <code>k</code> for the first time) is <code>i * (k - i)</code>.


## Deriving the probability of "stopping non-zero"

Let <code>prob_pos<sub>k</sub>(i)</code> denote the probability that the drunkard's walk started at integer <code>i</code> (</code>0 &le; i &le; k</code>) stops at <code>k</code> before ever visiting <code>0</code>.

For any <code>i</code> with <code>0&lt;i&lt;k</code> we can expand <code>prob_pos<sub>k</sub>(i)</code> by one walk step to get: <code>prob_pos<sub>k</sub>(i) = (1/2) prob_pos<sub>k</sub>(i-1) + (1/2) prob_pos<sub>k</sub>(i+1)</code>. It is just a matter of algebra to check that <code>prob_pos<sub>k</sub>(i) = i/k</code> obeys this recurrence, and has the desired values for the boundary <code>i = 0, k</code>.

As a warm up we confirm the claim.


In [1]:
import sympy

i, k = sympy.symbols("i k")

check = sympy.expand(
    ((1/2) * (i-1)/k + (1/2) * (i+1)/k) 
    - i/k)
assert check == 0

print(check)


0


## Deriving the "expected time to stop"

Let <code>e_time<sub>k</sub>(i)</code> denote the expected number of steps the drunkard's walk started at integer <code>i</code> (</code>0 &le; i &le; k</code>) takes to reach *either* of <code>0</code> or <code>k</code> for the first time. We are not yet specifying which one is first reached.

For any <code>i</code> with <code>0 &lt; i &lt; k</code> we can expand <code>e_time<sub>k</sub>(i)</code> by one walk step to get: <code>e_time<sub>k</sub>(i) = 1 + (1/2) e_time<sub>k</sub>(i-1) + (1/2) e_time<sub>k</sub>(i+1)</code>. It is again, a matter of  algebra to check that <code>e_time<sub>k</sub>(i) = i * (k-i)</code> obeys this recurrence, and has the desired values on the boundary.

Let's confirm this claim.


In [2]:
check = sympy.expand(
    (1 + (1/2) * (i-1) * (k-(i-1)) + (1/2) * (i+1) * (k-(i+1)))
    - i * (k-i))
assert check == 0

print(check)


0


The above directly gives us the "expected time to step distance <code>d</code> is around <code>d**2</code> steps" observation without the usual appeal to linear of expectation applied to independent variances. Note: we the expected stopping time to hit <code>0</code> is going to often be very different than the time to hit <code>k</code> (and both roughly proportional to the square of how far the specified target is from the start position).


# Defining Markov chains

For our a given run of our random process call the sequence of states <code>s<sub>0</sub></code>, <code>s<sub>1</sub></code>, ... . Where:

  * All the <code>s<sub>j</sub></code> are integers in the range <code>0</code> through <code>k</code>.
  * If <code>s<sub>j</sub></code> isn't <code>0</code> or <code>k</code>, then <code>|s<sub>i</sub> - s<sub>j+1</sub>| = 1</code> (the process has not stopped)
  * If <code>s<sub>j</sub></code> is <code>0</code> or <code>k</code>, then <code>s<sub>j+1</sub> = s<sub>j</sub></code> (the chain has "stopped").

Let <code>S<sub>k</sub>(i)</code> denote the set of all possible sequences <code>s</code> of integers obeying the above rules where:

  * The first state <code>s<sub>1</sub> = i</code>.
  * The "probability" of <code>s</code> in <code>S<sub>k</sub>(i)</code> is equal to:
     * <code>P[s] = 1</code>, if <code>s<sub>j+1</sub> = s<sub>j</sub></code> for all <code>j</code>. Call such <code>s</code> "stopped."
     * <code>P[s] = 0</code>, if <code>s<sub>j+1</sub> != s<sub>j</sub></code> for all <code>j</code>. Call such <code>s</code> "not stopped."
     * <code>P[s] = 1/2<sup>j</code></code> where <code>j</code> is the highest index such that <code>s<sub>j+1</sub> != s<sub>j</sub></code>. Call such <code>s</code> "stopped."

On can check that the <code>P[s] &ge; 0</code> for all <code>s</code> in <code>S<sub>k</sub>(i)</code>, <code>sum<sub>s in S<sub>k</sub>(i)</sub> = 1</code>, and the non-stopped sequences have probability measure zero (so can be ignored in probability arguments).


<code>P[]</code> is computing probability over the spaces of all possible sequences <code>s<sub>i</sub></code> generated by picking increasing or decreasing our state (for non stopped sequences) with 50/50 probability. It is a subtle point: probabilities are a distributional property of the family of all Markov chains, not of any *one* observed realization.

The above random process <code>S<sub>k</sub>(i)</code> has "the Markov property": history becomes irrelevant. That is:

<code>
<pre>
   P[s<sub>i+1</sub> = v | s<sub>1</sub>=v<sub>1</sub>, ..., s<sub>i</sub>=v<sub>i</sub>] = P[s<sub>i</sub> = v | s<sub>i</sub>=v<sub>i</sub>]
</pre>
</code>.

We often use the Markov property in analysis as follows: we can always pretend we are at the first move in a sequence! <code>P[s<sub>i+1</sub> = v<sub>2</sub> | s<sub>i</sub> = v<sub>1</sub>] = P[s<sub>2</sub> = v<sub>2</sub> | s<sub>1</sub> = v<sub>1</sub>]</code> always. 


